In [21]:
import smtplib
from email.mime.text import MIMEText
import imaplib
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import email
from datetime import datetime
import time
import re
import pandas as pd
import numpy as np
from datetime import date
from datetime import timedelta
from IPython.display import clear_output
from email.mime.application import MIMEApplication
import os
folder_path = r"/Users/olivia/Desktop"
os.chdir(folder_path)

### General functions

In [22]:
def Search_in(keyword, universe):
    """
    search 'keyword' from 'universe'
    ignore capital
    """
    return bool(re.search(keyword, universe, re.IGNORECASE))

In [23]:
def Email_sending(
    content = None,
    receivers=None,
    mail_host = 'smtp.office365.com',
    mail_user = '***@finexisam.com',
    mail_pass = '***',
    sender = '***@finexisam.com',
):
    """
    send 'content' to 'receiver'
    """
    message = MIMEText(subject,'plain','utf-8')
    message['Subject'] = subject
    message['From'] = sender 
    message['To'] = receivers[0]
    try:
        smtpObj = smtplib.SMTP_SSL(mail_host,465) 
#         smtpObj.connect(mail_host,465)
#         smtpObj.ehlo()
#         smtpObj.starttls()
#         smtpObj.ehlo()
        smtpObj.login(mail_user,mail_pass) 
        smtpObj.sendmail(sender,receivers,content) 
        smtpObj.quit()
        print('Sent successfully...')
    except Exception as e:
        print("Exception : {}".format(e))

In [5]:
## testing......
# mail_host = "smtp.gmail.com"
# mail_user = "olivia19931019@gmail.com"
# mail_pass = "Zhous_8875"
# receivers = ["luge1123@gmail.com", "olivia19931019@gmail.com"]
# sender =  mail_user

# content = "Love never fails!"


# Email_sending(
#     content=content, 
#     receivers=receivers,
#     mail_host = mail_host,
#     mail_user = mail_user,
#     mail_pass =mail_pass,
#     sender = sender,
# )

Sent successfully...


In [24]:
def convert_str_to_datetime(date_str):
    """
    convert date from string to datetime format
    """
    if type(date_str)==str:
        try:
            date_str = datetime.strptime(date_str, '%m/%d/%Y')
        except:
            try:
                date_str = datetime.strptime(date_str, '%Y-%m-%d')
            except:
                print(date_str, " cannot convert to datetime")
    return date_str

In [25]:
def convert_datetime_to_str(date_time):
    """
    convert date from datetime format to string
    """
    if type(date_time)!=str:
        try:
            date_time = date_time.strftime("%m/%d/%Y")
        except:
            print('oops...')

    return date_time

In [26]:
def convert_table_format(DF):
    DF = DF.reset_index(drop=True)
    for i in DF.index:
        DF.loc[i, 'Date'] = convert_datetime_to_str(DF.loc[i, 'Date'])
    return DF

# DF['Date'].apply(convert_datetime_to_str)

### FX1

In [27]:
def email_saving(typ, data, folder_path):
    folder_record = os.path.join(folder_path , "FX1 Record")
    emailbody = data[0][1]
    mail = email.message_from_bytes(emailbody)
    fileName = None 
    # Retrieve attachment file name
    for part in mail.walk():
        if part.get_content_maintype() == 'multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue
            
        fileName = part.get_filename()  

    # decode
        try:
            fileName = decode_header(fileName)[0][0].decode(decode_header(fileName)[0][1])
        except:
            pass
        
    # save attachment
        if (fileName != None) & ('jpg' not in fileName):
                    
                filePath = os.path.join(folder_record, fileName)
                if not os.path.isfile(filePath):
                    fp = open(filePath, 'wb')
                    fp.write(part.get_payload(decode=True))
                    fp.close()
                else:
                    print("Existing File:" + fileName)
                    Email_sending("Duplicate: " + fileName, ["michael.ng@finexisam.com"])
                        
                        
    return fileName

In [15]:
def Check_email_FX(first_email_FX, folder_path, mail_host = 'smtp.office365.com', mail_user = '***@finexisam.com',
                   mail_pass = '***'):

    file_name = None
            
    # Email Login
    smtpObj = smtplib.SMTP(mail_host,25) 
    smtpObj.connect(mail_host,587)
    smtpObj.ehlo()
    smtpObj.starttls()
    smtpObj.ehlo()
    smtpObj.login(mail_user,mail_pass)  
    
    # Inbox Scanning
    client = imaplib.IMAP4_SSL(mail_host)
    client.login(mail_user, mail_pass)
    status, messages = client.select('INBOX')
    
    # Inbox Index
    type, data = client.search(None, 'ALL')
    mail_ids = data[0]
    id_list = mail_ids.split()   
    latest_email_FX = int(id_list[-1])

    #  Email
    for i in range(first_email_FX + 1, latest_email_FX + 1):
        typ, data = client.fetch(str(i), '(RFC822)' )
        for response_part in data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])
                email_subject = msg['subject']
                email_from = msg['from']

                
                # Check KGI Report
                if (Search_in("Summary", email_subject)) &(Search_in("Account", email_subject)) &(Search_in("kgi", email_from))\
                        & (Search_in("re", email_subject)==False) & (Search_in("fw", email_subject)==False):
                    file_name = email_saving(typ, data,folder_path)
    
    return file_name, latest_email_FX

In [16]:
def FX_FT_process(first_email_FX, folder_path, path_IB):
    time_now = str(datetime.now())[11:16]
    date_now = str(datetime.now())[:10]
    week_now = datetime.now().weekday()
    
    if week_now < 6:   # Weekdays
        
        if week_now > 0:
            # FX1 processing
            if (time_now>"10:00") & (time_now<"10:30"):
                print(date_now)
                file_name, first_email_FX = Check_email_FX(first_email_FX, folder_path)
                print(file_name)
                print(first_email_FX)
                FX1_record = Record_append_FX1(folder_path, file_name)
                generate_upload_file_FX1(folder_path, FX1_record)
                time.sleep(60*30)

            # FX1 status check
            elif (time_now>"10:30") & (time_now<"11:30"):
                FX1_record_check = pd.read_excel(folder_path + "\\FX1 Record\\FX1 Record.xlsx")
                if (datetime.today() -convert_datetime(list(FX1_record_check["Date"])[-1])).days == int(np.where(week_now == 0,3,1)):
                    print("Upload file prepared successfully!")
                    Email_sending("Upload file prepared successfully!", ["michael.ng@finexisam.com"])
                else:
                    print("Problem with upload file, please check!")
                    Email_sending("Problem with upload file, please check!", ["michael.ng@finexisam.com"])
                print("\n")
                time.sleep(60*60*4)
            else:
                pass
        else:
            pass
            
        
    else:    # Weekends
        pass

    return first_email_FX

### Auto Process

In [17]:
first_email_FX = 13056
while True:
    first_email_FX = FX_FT_process(first_email_FX, folder_path,path_IB)
    print(first_email_FX)
    time.sleep(20*60)

NameError: name 'path_IB' is not defined

In [19]:
help(time.sleep())

TypeError: sleep() takes exactly one argument (0 given)